In [8]:

import os, json
from src.utils import read_voting, VotingSchemas
import numpy as np
from src.outcomes import plurality_outcome, veto_outcome, borda_outcome, for_two_outcome
from src.optimized_code import plurality_outcome_op,veto_outcome_op, borda_outcome_op, for_two_outcome_op, happiness_level_total, compute_risk, happiness_level
from src.happiness_level import HappinessLevel
from src.strategic_voting_risk import StrategicVoting
from typing import Callable, Tuple, List
from itertools import permutations
import pandas as pd

import warnings

warnings.filterwarnings("ignore")

In [9]:
def read_voting_(file_path: str, table_name: str = "voting"):
    with open(file_path) as f:
        voting = json.load(f)[table_name]
        voting = np.array(voting)

    return voting

In [10]:
path_voting = "input/voting_result.json"
if "env" in os.environ:
    if os.environ["env"] == "docker":
        path_voting = "/input/voting_result.json"
        
voting = read_voting(path_voting, table_name="five_candidates")[:,:5]
voting_optimized = read_voting_(path_voting, table_name="five_candidates")[:,:5]
pd.DataFrame(voting_optimized)

,0,1,2,3,4
0,2,0,1,4,2
1,3,1,2,1,0
2,4,4,3,2,4
3,0,2,4,0,1
4,1,3,0,3,3


In [11]:
schemes = ["PLURALITY", "VETO", "BORDA", "FOR_TWO"]
outcomes = [plurality_outcome, veto_outcome, borda_outcome, for_two_outcome]
outcomes_op = [plurality_outcome_op, veto_outcome_op, borda_outcome_op, for_two_outcome_op]

if all(outcome_op(voting_optimized) == outcome(voting).winner for outcome_op, outcome in zip(outcomes_op, outcomes)):
    print("Winners match for all schemes.")

if all(np.array_equal(happiness_level_total(voting_optimized, outcome_op(voting_optimized)),
                      HappinessLevel(voting, outcome(voting), scheme).voter) for scheme, outcome_op, outcome in zip(schemes, outcomes_op, outcomes)):
    print("Happiness levels match for all schemes.")

if all(StrategicVoting(voting, HappinessLevel(voting, outcome(voting), scheme), outcome).run().risk == compute_risk(voting_optimized, outcome_op) for scheme, outcome_op, outcome in zip(schemes, outcomes_op, outcomes)):
    print("Risks match for all schemes.")


Winners match for all schemes.
Happiness levels match for all schemes.
Risks match for all schemes.


In [12]:
def compute_voter_risk_combinations(preferences: np.ndarray, 
                                    result: int, 
                                    initial_happinesses: np.ndarray, 
                                    i: int, 
                                    schema_outcome_f: Callable) -> Tuple[float, pd.DataFrame]:
    
    initial_happiness = initial_happinesses[i]
    initial_overall_happiness = initial_happiness.sum()
    vwr_idx = np.flatnonzero(preferences[:, i] == result)
    if vwr_idx.size == 0:
        return 0, pd.DataFrame(columns=["voter", "combination", "new_result", "strategic_happiness", "old_happiness", "overall_happiness", "previous_overall_happiness"])
    
    vwr = vwr_idx[0]
    best_happiness = initial_happiness
    strategies_happiness = pd.DataFrame(columns=["voter", "combination", "new_result", "strategic_happiness", "old_happiness", "overall_happiness", "previous_overall_happiness"])
    
    for perm in permutations(preferences[:, i]):
        idx = np.where(perm == result)[0][0]
        if idx >= vwr:
            new_voting = preferences.copy()
            new_voting[:, i] = perm
            new_result = schema_outcome_f(new_voting)
            new_happiness = happiness_level(preferences, i, new_result)
            
            if isinstance(new_happiness, (int, float)):
                new_happiness = np.array([new_happiness])
            
            new_overall_happiness = new_happiness.sum()
            
            if new_happiness > initial_happiness:
                strategies_happiness.loc[len(strategies_happiness)] = [i, perm, new_result, new_happiness, initial_happiness, new_overall_happiness, initial_overall_happiness]
                
            if new_happiness >= 1:  # Maximum happiness achieved
                return new_happiness - initial_happiness, strategies_happiness
            
            if new_happiness > best_happiness:
                best_happiness = new_happiness
    
    return best_happiness - initial_happiness, strategies_happiness

def compute_risk_combinations(preferences: np.ndarray, 
                              schema_outcome_f: Callable) -> Tuple[float, pd.DataFrame]:
    
    result = schema_outcome_f(preferences)
    initial_happinesses = happiness_level_total(preferences, result)
    num_unhappy_voters = np.count_nonzero(initial_happinesses != 1)
    
    permutations_list = []
    total_risk = 0
    
    for voter in range(preferences.shape[1]):
        risk, permutations_voter = compute_voter_risk_combinations(preferences, result, initial_happinesses, voter, schema_outcome_f) 
        permutations_list.append(permutations_voter)
        total_risk += risk
    
    permutations = pd.concat(permutations_list, ignore_index=True)
    
    if total_risk == 0:
        return 0, permutations
    
    return total_risk / num_unhappy_voters, permutations


In [13]:
display(StrategicVoting(voting,HappinessLevel(voting,plurality_outcome(voting),"PLURALITY"),plurality_outcome).run().all)
display(compute_risk_combinations(voting_optimized,plurality_outcome_op))

,voter,strategic_voting,new_result,strategic_H,previous_H,strategic_overall_H,previous_overall_H
0,1,"[1, 0, 4, 2, 3]",1,0.640985,0.359015,2.640985,3.5
1,1,"[1, 0, 4, 3, 2]",1,0.640985,0.359015,2.640985,3.5
2,1,"[1, 0, 3, 4, 2]",1,0.640985,0.359015,2.640985,3.5
3,1,"[1, 0, 3, 2, 4]",1,0.640985,0.359015,2.640985,3.5
4,1,"[1, 4, 0, 2, 3]",1,0.640985,0.359015,2.640985,3.5
5,1,"[1, 4, 0, 3, 2]",1,0.640985,0.359015,2.640985,3.5
6,1,"[1, 4, 3, 0, 2]",1,0.640985,0.359015,2.640985,3.5
7,1,"[1, 4, 3, 2, 0]",1,0.640985,0.359015,2.640985,3.5
8,1,"[1, 3, 0, 4, 2]",1,0.640985,0.359015,2.640985,3.5
9,1,"[1, 3, 0, 2, 4]",1,0.640985,0.359015,2.640985,3.5


(array([0.14098507]),
    voter      combination new_result   strategic_happiness  old_happiness  \
 0      1  (1, 0, 4, 2, 3)          1  [0.6409850721733619]       0.359015   
 1      1  (1, 0, 4, 3, 2)          1  [0.6409850721733619]       0.359015   
 2      1  (1, 0, 3, 4, 2)          1  [0.6409850721733619]       0.359015   
 3      1  (1, 0, 3, 2, 4)          1  [0.6409850721733619]       0.359015   
 4      1  (1, 4, 0, 2, 3)          1  [0.6409850721733619]       0.359015   
 5      1  (1, 4, 0, 3, 2)          1  [0.6409850721733619]       0.359015   
 6      1  (1, 4, 3, 0, 2)          1  [0.6409850721733619]       0.359015   
 7      1  (1, 4, 3, 2, 0)          1  [0.6409850721733619]       0.359015   
 8      1  (1, 3, 0, 4, 2)          1  [0.6409850721733619]       0.359015   
 9      1  (1, 3, 0, 2, 4)          1  [0.6409850721733619]       0.359015   
 10     1  (1, 3, 4, 0, 2)          1  [0.6409850721733619]       0.359015   
 11     1  (1, 3, 4, 2, 0)          1  [0.

In [14]:
%timeit -n 10 StrategicVoting(voting,HappinessLevel(voting,plurality_outcome(voting),"PLURALITY"),plurality_outcome).run().all
%timeit -n 10 compute_risk_combinations(voting_optimized,plurality_outcome_op)

34.5 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
13.5 ms ± 212 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
